In [1]:
from statistics import mean

import loralib as lora
import scipy.stats as stats
import torch
import torch.nn as nn
import wandb
from datasets import Dataset as ds
from datasets import load_dataset, load_from_disk
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    MT5EncoderModel,
    get_scheduler,
)
from transformers.modeling_outputs import BaseModelOutput

[2023-07-24 17:46:09,423] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
torch.cuda.mem_get_info()[0] / 1024**3

31.447509765625

In [5]:
model_encoder_name = "bigscience/mt0-large"

device = "cuda:0"

random_seed = 42

In [6]:
wandb.login()

True

# Load data

In [ ]:
dataset = load_dataset("RicardoRei/wmt-da-human-evaluation", split="train")

In [ ]:
dataset_train = dataset.filter(
    lambda example: (example["year"] != 2022)
    and (
        (example["lp"] == "en-ru")
        or (example["lp"] == "zh-en")
        or (example["lp"] == "en-de")
    )
)

In [ ]:
prompt_column = []

prompt_template = """
Score the following translation from {source_lang} to {target_lang} with respect to the human reference on a continuous scale from 0 to 100, where score of zero means "no meaning preserved" and score of one hundred means "perfect meaning and grammar".
{source_lang} source: "{source_seg}"
{target_lang} human reference: {reference_seg}
{target_lang} translation: "{target_seg}"
Score:
"""

for i in tqdm(range(len(dataset_train))):
    example = dataset_train[i]
    sl, tl = example["lp"].split("-")
    prompt_column.append(
        prompt_template.format(
            source_lang=sl,
            target_lang=tl,
            source_seg=example["src"],
            reference_seg=example["ref"],
            target_seg=example["mt"],
        )
    )

In [8]:
dataset_train = dataset_train.add_column(name="prompt", column=prompt_column)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/RicardoRei___csv/RicardoRei--wmt-da-human-evaluation-a4a96cd6106c3667/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-b815f5b3ffcfbe9a.arrow


In [9]:
dataset_train[41152]

{'lp': 'zh-en',
 'src': '在三年半后重新任职总统之前，普京先生担任俄罗斯总理一职。',
 'mt': 'Mr. Putin served as Russian prime minister before resuming his presidency Prime Minister of Russia after three and a half years in office.',
 'ref': 'Mr Putin became prime minister, before returning to the presidency just three-and-a-half years later.',
 'score': -0.048723632415222,
 'raw': 74.5,
 'annotators': 2,
 'domain': 'news',
 'year': 2017,
 'prompt': '\nScore the following translation from zh to en with respect to the human reference on a continuous scale from 0 to 100, where score of zero means "no meaning preserved" and score of one hundred means "perfect meaning and grammar".\nzh source: "在三年半后重新任职总统之前，普京先生担任俄罗斯总理一职。"\nen human reference: Mr Putin became prime minister, before returning to the presidency just three-and-a-half years later.\nen translation: "Mr. Putin served as Russian prime minister before resuming his presidency Prime Minister of Russia after three and a half years in office."\nScore:\n'}

# T5 tokenizer initialize

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_encoder_name)

# DataLoader

## Tokenize

In [11]:
dataset_tokenized = dataset_train.map(
    lambda example: tokenizer(
        example["prompt"],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    ),
    batched=True,
)

Map:   0%|          | 0/361129 [00:00<?, ? examples/s]

In [12]:
dataset_tokenized.save_to_disk("wmt-da_tokenized")

Saving the dataset (0/3 shards):   0%|          | 0/361129 [00:00<?, ? examples/s]

## Convert to DataLoader

In [8]:
dataset_tokenized = load_from_disk("wmt-da_tokenized")

In [9]:
dataset_tokenized = ds.from_dict(dataset_tokenized[:25000])

In [10]:
dataset_tokenized = (
    dataset_tokenized.with_format("torch")
    .remove_columns(
        ["lp", "src", "mt", "ref", "score", "annotators", "domain", "year", "prompt"]
    )
    .rename_column("raw", "label")
)

In [11]:
dataset_traineval = dataset_tokenized.train_test_split(test_size=0.2, seed=random_seed)

dataset_traineval

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [12]:
data_collactor = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
dataloader_train = DataLoader(
    dataset_traineval["train"], batch_size=8, shuffle=True, collate_fn=data_collactor
)

dataloader_eval = DataLoader(
    dataset_traineval["test"], batch_size=8, collate_fn=data_collactor
)

# Model

Tried achitectures:
1. Flan-T5-base-encoder + Dropout + MLP
2. Flan-T5-base-encoder + Dropout + MLP + Sigmoid
3. MT0-base-encoder + Dropout + MLP + Sigmoid
    1. Eval MSE: 1107
4. MT0-base-encoder + Dropout + MLP + Dropout + Sigmoid
    1. Eval MSE: 806
    2. Eval K-tau: 0.245
5. LoRA MT0-base-encoder + Dropout + MLP + Dropout + Sigmoid(2 epochs)
    1. Eval MSE: 780
    2. Eval K-tau: 0.183
6. LoRA MT0-large-encoder + Dropout + MLP + Dropout + Sigmoid(2 epochs)
    1. Eval MSE: 794
    2. Eval K-tau: 0.218

To check:

1. Different activations on last layer(e.g. Sigmoid)
2. Different activations on hidden layers
3. Different losses(e.g. RMSE)
4. Different hidden layers in MLP
5. More/less dropouts
6. Different batch size
7. Differen lr

To do:
1. Move .to(device) from train loop to tokenize
2. Use LoRA adapter
3. Use int8/int4
4. Replace quality metric with Kendal-tau and Spearman

Plan:

Add WandB and check train/loss function on big train dataset and base architecture(t5+dropout+mlp without dropouts, mse loss, no act on last layer) <br />
Do other experiments whether it converges or it doesn't

In [14]:
def mean_pooling(token_embeddings, attention_mask):
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

In [15]:
class T5Regressor(nn.Module):
    def __init__(self, checkpoint, sizes_mlp, act=nn.ReLU):
        super(T5Regressor, self).__init__()

        self.llm = MT5EncoderModel.from_pretrained(
            checkpoint, output_attentions=True, output_hidden_states=True
        )

        self.llm_output_shape = sizes_mlp[0]

        self.dropout = nn.Dropout(0.1)

        layers = []
        for i in range(len(sizes_mlp) - 1):
            # layers.append(nn.Linear(sizes_mlp[i], sizes_mlp[i + 1]))
            layers.append(lora.Linear(sizes_mlp[i], sizes_mlp[i + 1], r=16))
            if i < len(sizes_mlp) - 2:
                layers.append(act())

        layers.append(nn.Dropout(0.1))
        self.mlp = nn.Sequential(*layers)
        self.output_layer = nn.Sigmoid()

        self.loss_fc = nn.MSELoss()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.llm(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = mean_pooling(
            outputs.last_hidden_state, outputs.attentions[-1][:, 0, :, 0]
        )
        outputs_sequence = self.dropout(embeddings)

        logits = self.output_layer(self.mlp(outputs_sequence)) * 100

        loss = None
        if labels is not None:
            loss = self.loss_fc(logits.view(-1, 1), labels.view(-1).unsqueeze(1))

        return (
            BaseModelOutput(
                last_hidden_state=outputs.last_hidden_state,
                hidden_states=outputs.hidden_states,
                attentions=outputs.attentions,
            ),
            logits,
            loss,
        )

# Train

In [16]:
wandb.init(entity="airi23-efficient-llm-metrics", project="t5regressor")

wandb: Currently logged in as: egoluback (airi23-efficient-llm-metrics). Use `wandb login --relogin` to force relogin


In [18]:
model = T5Regressor(checkpoint=model_encoder_name, sizes_mlp=[1024, 192, 48, 1])

model.to(device)

T5Regressor(
  (llm): MT5EncoderModel(
    (shared): Embedding(250112, 1024)
    (encoder): MT5Stack(
      (embed_tokens): Embedding(250112, 1024)
      (block): ModuleList(
        (0): MT5Block(
          (layer): ModuleList(
            (0): MT5LayerSelfAttention(
              (SelfAttention): MT5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): MT5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): MT5LayerFF(
              (DenseReluDense): MT5DenseGatedActDense(
                (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
                (wi_1): L

In [19]:
# batch = {k: v.to(device) for k, v in next(iter(dataloader_train)).items()}

# with torch.no_grad():
#     outputs = model(**batch)

# outputs[1]

In [20]:
optimizer = AdamW(model.parameters(), lr=3e-4)

In [21]:
num_epochs = 2
num_training_steps = num_epochs * len(dataloader_train)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

5000


In [22]:
metric = nn.MSELoss()
# TODO: replace with Kendall-tau/Spearman

In [23]:
progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(dataloader_eval)))

print(f"Train size: {len(dataloader_train)}")
print(f"Eval size: {len(dataloader_eval)}")

lora.mark_only_lora_as_trainable(model)
for epoch in range(num_epochs):
    print(f"TRAIN EPOCH {epoch + 1}")
    model.train()
    for batch in dataloader_train:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        loss = outputs[2]
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        # progress_bar_train.set_postfix({"loss": loss.item()})
        progress_bar_train.set_postfix(
            {"loss": loss.item(), "logits": outputs[1][1].item()}
        )
        progress_bar_train.update(1)

        wandb.log({"loss": loss.item()})

    print("EVAL")
    model.eval()
    mse_metrics = []
    predicted = []
    labels = []
    for batch in dataloader_eval:
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs[1]

        mse_metric = metric(logits, batch["labels"]).item()
        for i in range(8):
            predicted.append(logits[i].item())
            labels.append(batch["labels"][i].item())
        mse_metrics.append(mse_metric)
        progress_bar_eval.set_postfix({"loss": mse_metric})
        progress_bar_eval.update(1)

    print(f"Eval MSE: {mean(mse_metrics)}")
    print(f"Eval Kendall tau-b: {stats.kendalltau(predicted, labels)[0]}")

  0%|          | 0/1250 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Train size: 2500
Eval size: 625
TRAIN EPOCH 1


 50%|█████     | 2500/5000 [13:41<13:22,  3.11it/s, loss=930, logits=62.1]    /home/user/conda/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


EVAL



 50%|█████     | 625/1250 [16:58<03:14,  3.22it/s, loss=693]

Eval MSE: 779.3868517822266
Eval Kendall tau-b: 0.2033833537324743
TRAIN EPOCH 2


100%|██████████| 5000/5000 [30:37<00:00,  3.13it/s, loss=1.24e+3, logits=61.7]   

EVAL



100%|██████████| 1250/1250 [33:55<00:00,  3.22it/s, loss=695]

Eval MSE: 794.5471818847657
Eval Kendall tau-b: 0.21893053023578973
